# Join and Wrangle
---------

### Author Information
**Author:** PJ Gibson  
**Email:** Peter.Gibson@doh.wa.gov  
**Github:**   https://github.com/DOH-PJG1303

### Project Information
**Created Date:** 2023-08-04  
**Last Updated:** 2023-08-08  
**Version:** 1  

### Description
To start off, we'll be wrangling raw data output from the [Synthetic-Gold](https://github.com/DOH-PJG1303/Synthetic-Gold) github project.
Please explore that documentation if you are curious about the project and the underlying data.
We ran the simulation for the state of Nebraska, so all data is relevant to that state.
To manage the size of the data we'll have publicly stored on Github, we only captured relevant data for each table for the population living in years 2019-2022

1. We'll combine raw data tables from the synthetic gold database to extract as much information as we might want.
    - **NOTE:** We've filtered the data to only people born in "Hall County, NE" to reduce the volume size.  Most computers wouldn't be able to handle the computation of much larger datasets.
2. Then we'll intentionally tweak some fields to add congregate living facility phone numbers as well as nicknames for a subset of the population.  These serve as extra fields to then be called on when we apply messiness to data. 

### Notes

## 1. Setup

In [1]:
import pandas as pd
import numpy as np

## 2. Load data

In [2]:
# Load in our "long-formatted" data.  Note only years 2019-2022
df_long = pd.read_parquet('../../Data/SyntheticGold/LongSyntheticGold')

# Load in births data, specifically to catch parents_partnership_id as well as status as twin/triplet/jr
df_births = pd.read_parquet('../../Data/SyntheticGold/Births')[['ssn','parents_partnership_id','is_twin','is_triplet','is_jr']]

# Read in housing events to track births (event=1)
df_housingEvents = pd.read_parquet('../../Data/SyntheticGold/HousingEvents').query('event_type == 1')[['ssn','new_house_id']]

# Read in email events to track secondary email
df_emailEvents = pd.read_parquet('../../Data/SyntheticGold/EmailEvents')[['ssn','secondary_email','event_year']]

In [3]:
# Read in housing lookup.  We'll later divide this into 2 dataframes
df_housingLookup = pd.read_parquet('../../Data/SyntheticGold/HousingLookup')

# Get a list of houses in Hall County
df_county_houses = df_housingLookup.query('county_name == "Hall County"')[['house_id']]

# Filter down our columns
df_housingLookup = df_housingLookup[['house_id','building_id','num_units']]

In [4]:
df_nicknames = pd.read_csv('../../Data/SyntheticGold/Nicknames.csv')

## 3. Join Data

In [5]:
# Find SSNs belonging to all people born in "Hall County"
df = pd.merge(df_housingEvents, df_county_houses, left_on='new_house_id', right_on='house_id', how='inner')[['ssn']]

# Collect long-formatted synthetic data for all those people
df = pd.merge(df, df_long, on='ssn', how='inner')

# Add "building_id","num_units" columns to existing data
df = pd.merge(df, df_housingLookup, on='house_id', how='inner')

# Add 'parents_partnership_id','is_twin','is_triplet','is_jr' columns to existing data
df = pd.merge(df, df_births, on='ssn', how='inner')

# Add 'secondary_email','event_year' columns to existing data
df = pd.merge(df, df_emailEvents, on='ssn', how='left')

# Only provide secondary email for year's where that data would have been available
mask_email = df['event_year'] > df['year']
df.loc[mask_email,'secondary_email'] = np.nan

## 4. Selective Data Wrangling

### 4.1 Congregate phone numbers

Based on the constraints I had while creating the synthetic society, phone numbers serve as very good unique identifiers for people.
Because of this, I'd like to create some logic that makes phone numbers less of unique identifiers and more representative of what we'd see in real life.
To do this, we'll do the following:

1. If you live in a building with >75 units, you get the same landline phone number (taken from one person within building, smallest phone number numerically (sorted ascending)).
2. A person's phone number is taken as `COALESCE( congregate_landline, mobile_phone, landline )`

In [6]:
# For every building with 75+ units, get 1 representative unit's landline in a dataframe with ['building_id','phone_landline_congregate']
df_congregate_facilities = df[df['num_units'] > 75]\
                                .sort_values('phone_landline', na_position='last')\
                                .groupby('building_id')\
                                .first()\
                                .rename(columns={'phone_landline':'phone_landline_congregate'})\
                                .reset_index()\
                                [['building_id','phone_landline_congregate']]

# Join up the data
df = pd.merge(df, df_congregate_facilities, on='building_id', how='left')

# Assign phone number
df = df.assign(phone = df[['phone_landline_congregate', 'phone_mobile', 'phone_landline']].bfill(axis=1).iloc[:, 0])

### 4.2 Nicknames

We want to give nicknames to some percent of the population.
For reproducibility's sake, I chose the smallest SSN value per group of offspring (share same parents).
At one point this represented the first born, but for all births post-2011, this might not be the case.

For this specific data, around 12.2% of the population will have a nickname.
This nickname is only activated by the nickname() function within the "DirtyFunctions".

In [7]:
# Get name,ssn of one person per family (smallest SSN value)
df_firstborns = df.assign(dedupe_col1=df.groupby('parents_partnership_id')['ssn'].rank(method='min'))\
                    .query('dedupe_col1 == 1')\
                    .drop(['dedupe_col1'], axis=1)\
                    .drop_duplicates(subset=['ssn'])\
                    [['ssn','first_name']]

# Assign all relevant firstname nicknames, random shuffle (w/ random state), and drop duplicates to randomly select one nickname when multiple available.
df_firstborn_nicknames = pd.merge(df_firstborns, df_nicknames, on='first_name', how='left')\
                           .sample(frac=1, random_state=42)\
                           .drop_duplicates(subset=['ssn'])\
                           [['ssn','nickname']]

# Assign those nicknames
df = pd.merge(df, df_firstborn_nicknames, on='ssn', how='left')

### 4.3 Unique ID

In [8]:
df = df.assign(unique_id = np.arange(len(df)))

### 4.4 Person ID

Since we'll be using the SSN field as a field for linking, we should assign a copy of the SSN value (untouched) as the person_id.

In [9]:
df['person_id'] = df['ssn'].copy()

## 5. Save

In [11]:
df.to_parquet('../../Data/Training/01. Wrangled Clean Data.parquet', index=False)